<a href="https://colab.research.google.com/github/obengdouglas/DSI-2022/blob/main/Face_mask_Detect_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# <center>[The twist Challenge - DSI 2022 Module 1B](http://dsi-program.com/)
> <center>Train and deploy a computer vision model</center>

<center><img src="https://venturebeat.com/wp-content/uploads/2017/01/computer-vision.jpg?fit=750%2C469&strip=all" ></center>

<center><h1>Report: Douglas Obeng</h1></center>

<h1>Introduction</h1>
    
🐠Goals of the Twist Challenge

- Build a machine vision app to solve some “interesting” problem* 
- Get your own training data by scraping it from the web or using an API 
- You are encouraged to use pretrained vision models and fine-tune them (transfer learning) to better solve your problem
- Build a GUI that can demo your classifier in real time on new images

🐠 Presentation should do the following

- Explain the problem you are solving and why you chose it (NB: 80/20 rule)
- Explain how you solved it (what model, what data, how you got your data and implemented the GUI etc…)
- Do a Live demo of your tool working 



# Scrape data From the web
- Using BeautifulSoup

In [ ]:
import urllib
# from urllib.request import urlopen, urlretrieve, quote
# from urllib.parse import urljoin
import bs4
from bs4 import BeautifulSoup as bs
import requests
import re
import requests, os, string

url = 'https://makeml.app/datasets/mask'
response = urllib.request.urlopen(url)
html = response.read()

soup = bs4.BeautifulSoup(html, features="lxml")
a = soup.find('a')
onclick = a.attrs['onclick']
left = onclick.find("open('")
right = onclick.find(".zip?",left+1)
print('URL is: {}'.format(onclick[left+6:right+4]))

URL is: https://arcraftimages.s3-accelerate.amazonaws.com/Datasets/Mask/MaskPascalVOC.zip


Follow link in above result to download data

Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns
import ast
import time
from tqdm.notebook import tqdm
tqdm.pandas()
import glob
import shutil
import sys
sys.path.append('/content/drive/MyDrive/Colab_Data/face-mask-detection')
import torch
from PIL import Image
from joblib import Parallel, delayed
from IPython.display import display

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

# 3. Train

<p align=""><a ref=ultralytics"><img width="1000" src="https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/615627e5824c9c6195abfda9_computer-vision-cycle.png"/></a></p>



In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-248-gcb2ad9f torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.2/78.2 GB disk)


In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

wandb: Currently logged in as: obengdouglas (use `wandb login --relogin` to force relogin)


True

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 8 --epochs 16 --data /content/drive/MyDrive/Colab_Data/face-mask-detection/custom_facemask.yaml --weights yolov5s.pt --cache

wandb: Currently logged in as: obengdouglas (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/Colab_Data/face-mask-detection/custom_facemask.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=16, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-248-gcb2ad9f torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cl

# 1. Inference



In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source /content/drive/MyDrive/Colab_Data/face-mask-detection/images/test


detect: weights=['runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/Colab_Data/face-mask-detection/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-248-gcb2ad9f torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/84 /content/drive/.shortcut-targets-by-id/1JrzNTOP9woECJL-FxjOqkf1slRlt6BHS/face-mask-detection/images/test/maksssksksss102_png.rf.a93f2ae33f1c8f64822395ac78b3630f.jpg: 640x640 2 with masks, Done. (0.034s)
image 2/84 /content/drive/.shortcut-targets-by-id/1JrzNTOP9woECJL-FxjOqkf1slRlt6BHS/face-mask-detection/images/te

# 4. Visualize

In [ ]:
# Evolve
# !python train.py --img 640 --batch 64 --epochs 100 --data coco128.yaml --weights yolov5s.pt --cache --noautoanchor --evolve
# !d=runs/train/evolve && cp evolve.* $d && zip -r evolve.zip $d && gsutil mv evolve.zip gs://bucket  # upload results (optional)

In [ ]:
pip install gradio

     |████████████████████████████████| 871 kB 6.2 MB/s 
     |████████████████████████████████| 2.0 MB 36.5 MB/s 
     |████████████████████████████████| 1.1 MB 41.6 MB/s 
     |████████████████████████████████| 52 kB 853 kB/s 
     |████████████████████████████████| 52 kB 1.0 MB/s 
     |████████████████████████████████| 210 kB 42.4 MB/s 
     |████████████████████████████████| 144 kB 40.7 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 43.5 MB/s 
     |████████████████████████████████| 10.9 MB 31.6 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 61 kB 419 kB/s 
     |████████████████████████████████| 856 kB 39.6 MB/s 
     |████████████████████████████████| 3.6 MB 25.2 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=9336

In [ ]:
import gradio as gr

best = /content/yolov5/runs/train/exp/weights/best.pt
def video_flip(video):
    return video


best.pt = gr.Interface(video_flip, gr.inputs.Video(source="webcam"), "playable_video")

best.pt.launch()

In [29]:
while True:pass

KeyboardInterrupt: ignored

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()